<a href="https://colab.research.google.com/github/nmurillon/DeepLearningChallenge/blob/main/ZZ3-DL-GRAVEGEAL-MURILLON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports


In [11]:
import tensorflow as tf
import pandas as pd
import numpy as np
import time
from PIL import Image
import glob
import matplotlib.pyplot as plt
%matplotlib inline

# Ajouter les imports nécessaires
import os #Changer le répertoire courant pour les chemins des fichiers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

##Variables globales

In [2]:
num_epochs = 100
batch_size = 250

img_shape = (28,28,1)
img_size = img_shape[:2]

num_classes = 5
stop_freeze = 4


##Données

In [3]:
#On monte le drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
#on change le répertoire courant pour faciliter l'accès aux données
os.chdir('/content/drive/MyDrive/DeepLearning/') 

In [ ]:
classes = ["basket","eye","binoculars","rabbit","hand"]

train = pd.read_csv('train.csv')
valid = pd.read_csv('valid.csv')

In [ ]:
plt.figure(figsize=(18,9))
for i in range(0,5):
    ax= plt.subplot(3,2 ,i+1)
    mydata = pd.read_csv("train.csv",skiprows = [1], nrows=1)
    im = Image.open('images/'+classes[i]+'/'+ os.listdir("images/"+classes[i])[0])
    fig=ax.imshow(im)
    plt.title(classes[i])
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
plt.show()    


In [12]:
def Data():
    x_train, x_test, y_train, y_test = np.array([]), np.array([]), np.array([]), np.array([])

    for img_path in train['relative_path']:
      np.append(x_train, np.array(Image.open(img)), axis=0)

    for label in train['class_label']:
      np.append(y_train, label)

    for img_path in valid['relative_path']:
      np.append(x_test, np.array(Image.open(img)), axis=0)

    for label in valid['class_label']:
      np.append(y_test, label)

    x_train = x_train.reshape(x_train.shape[0], img_size, img_size, 1)/255
    x_test = x_test.reshape(x_test.shape[0], img_size, img_size, 1)/255

    # Conversion en matrices binaires
    y_train = tf.keras.utils.to_categorical(y_train, num_classes)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes)

    return ((x_train, y_train),(x_test, y_test))

##Modèle 1 : Fine tuning

Dans un premier temps, nous utiliserons le transfer Learning

In [ ]:
def TransferModel(input_shape,stop_freeze, model):
    '''
    @param model is a model from tensorflow.keras.applications
    '''
    pretrained_model = MobileNetV2(input_shape=image_shape)
    
    # On ôte la dernière couche de classification
    pretrained_model.layers.pop()   
    
    inputs = Input(input_shape)
    x = pretrained_model(inputs)
    # On fige tous les poids sauf ceux des stop_freeze dernières couches
    for i in range(len(pretrained_model.layers) - stop_freeze):
      pretrained_model.layers[i].trainable = False
    
    model = Sequential([pretrained_model,
                        Dense(num_classes, activation='softmax')])
    
    return model

##Entraînement du réseau

In [ ]:
#Choix du modèle pré-entraîné
pretrained_model = tf.keras.applications.MobileNetV2# TODO
pretrained_model_name = "MobileNetV2"

In [ ]:
(x_train, y_train) , (x_test, y_test) = Data()

model = TransferModel(img_shape, stop_freeze, pretrained_model)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Callback pour la sauvegarde du meilleur modèle
if not os.path.isdir("sauve"):
    os.mkdir("sauve")
    
checkpoint = ModelCheckpoint(f"sauve/{pretrained_model_name}-loss-{val_loss:.2f}-acc-{accuracy:.2f}.h5",
                                save_best_only=True,verbose=1)


# TODO !!!
train_steps_epoch = np.ceil(x_train.samples / batch_size)
val_steps_epoch = np.ceil(x_test.samples / batch_size)

# Entraînement
model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size,
                        epochs=num_epochs, verbose=1, callbacks=[checkpoint])

In [ ]:
#Construction du meilleur modèle
model = TransferModel(input_shape=image_shape,stop_freeze=2 )
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.load_weights("sauve/MobileNetV2_flowers-loss-0.76-acc-0.94.h5") # TODO

# Evaluation du modèle
evaluation = model.evaluate(x_test, verbose=1)
print("Perte = {0:5.3f}, Précision = {1:5.3f}".format(evaluation[0], evaluation[1]))

In [ ]:
# Quelques exemples de bonne / mauvaise classification
image_batch, label_batch = next(iter(x_test))

label_batch = [classes[np.argmax(label_batch[i])] for i in range(batch_size)]
predicted_classes = model.predict(image_batch)

predicted_ids = [np.argmax(predicted_classes[i]) for i in range(batch_size)]
predicted_classes = np.array([classes[id] for id in predicted_ids])

plt.figure(figsize=(10,9))
for n in range(30):
    plt.subplot(6,5,n+1)
    plt.subplots_adjust(hspace = 0.3)
    plt.imshow(image_batch[n])
    if predicted_classes[n] == label_batch[n]:
        color = "blue"
        title = predicted_classes[n].title()
    else:
        color = "red"
        title = f"{predicted_classes[n].title()}, correct:{label_batch[n]}"
    plt.title(title, color=color)
    plt.axis('off')
_ = plt.suptitle("Prédiction (bleu : OK, rouge : KO)")
plt.tight_layout()
plt.savefig("res.png",dpi=100)